### notebook to test FC Layer implementation after VGG and Resnet Layers

In [7]:
# import the torch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import torchvision.models as models

import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

# import custom functions
from helper_functions import *
from latency_helpers import *
from loading_functions import *


In [9]:
feat_folder = '../../Features/'
feat_name = 'SPEC'
seg_len = 20
n_per_seg = 1024
interferences = ['WIFI','CLEAN','BLUE','BOTH']
output_name = 'drones'
feat_format = 'IMG'

dataset = DroneDetectTorch(feat_folder, feat_name, seg_len, n_per_seg, feat_format,
                                output_name, interferences)

Directory Name:  ../../Features/IMG_SPEC_1024_20/


In [30]:
class VGGFC(nn.Module):
    def __init__(self, num_classes):
        super(VGGFC,self).__init__()
        self.num_classes = num_classes
        self.vggfull = models.vgg16(pretrained=True)
        modules=list(self.vggfull.children())[:-1] # remove the fully connected layer & adaptive averaging
        self.vggfeats=nn.Sequential(*modules)
        
        for param in self.vggfeats.parameters():
            param.requires_grad_(False)
        
        self._fc = nn.Linear(25088, num_classes)
    def forward(self, x):
        x = torch.moveaxis(x, 2, 0)
        x = self.vggfeats(x)
        x = x.flatten()
        x = self._fc(x)
        
        return x

In [33]:
Model = VGGFC(7)

In [34]:
sample = dataset.__getitem__(12)[0]
# sample2 = torch.moveaxis(sample, 2, 0)
Model(sample)

tensor([ 0.0830, -0.1303, -0.2537,  0.2783, -0.0017, -0.1413,  0.1179],
       grad_fn=<AddBackward0>)

In [18]:
sample2.shape

torch.Size([3, 224, 224])

In [2]:
class ResnetFC(nn.Module):
    def __init__(self, num_classes):
        super(ResnetFC,self).__init__()
        self.num_classes = num_classes
        self.resnetfull = models.resnet50(pretrained=True)
        modules=list(self.resnetfull.children())[:-2] # remove the fully connected layer & adaptive averaging
        self.resnetfeats=nn.Sequential(*modules)
        
        for param in self.resnetfeats.parameters():
            self.resnetfeats.requires_grad_(False)
        
        self._fc = nn.Linear(1505280, num_classes)
    def forward(self, x):
#         batch_size ,_,_ =x.shape
        
        # replicate the image to have 3 channels
        x = x.repeat(1,3,1,1)
        print(x.shape)
        x = self.resnetfeats(x)
        x = x.flatten()
        print(x.shape)
        x = self._fc(x)
        
        return x